In [2]:
# 비디오 확장자(mp4, mov)

# 이미지 데이터
- 이미지 총 245 장

In [3]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

In [4]:
# 데이터 경로
ROOT_DIR = r"C:/Users/yjneo/workspace/hecto_deepfake\data/test_data"

# 이미지 확장자 (jpg, png, jpeg, jfif,)
IMG_EXTENSIONS = (".jpg", ".jpeg", "png","jfif")

image_paths = []
for root, _, files in os.walk(ROOT_DIR):
    for f in files:
        if f.lower().endswith(IMG_EXTENSIONS):
            image_paths.append(os.path.join(root, f))

print(f"Total images found: {len(image_paths)}")


Total images found: 245


In [5]:
records = []

for path in tqdm(image_paths):
    try: 
        with Image.open(path) as img:
            img = img.convert('RGB')
            w,h = img.size 
            aspect_ratio = w/h

            img_np = np.array(img)
            mean_brightness = img_np.mean()

            file_size_kb = os.path.getsize(path) / 1024

            records.append({
                "path" : path,
                "width" : w,
                "height":h,
                "aspect_ratio" : round(aspect_ratio,3),
                "brightness": round(mean_brightness,2),
                "file_size_kb": round(file_size_kb,2),
                "format":img.format

            })
    except Exception as e:
        print(f'Failed to process {path} : {e}')

df = pd.DataFrame(records)
print(df.head())

100%|██████████| 245/245 [00:30<00:00,  8.05it/s]


                                                path  width  height  \
0  C:/Users/yjneo/workspace/hecto_deepfake\data/t...   2472    4160   
1  C:/Users/yjneo/workspace/hecto_deepfake\data/t...   4128    2752   
2  C:/Users/yjneo/workspace/hecto_deepfake\data/t...   3024    4032   
3  C:/Users/yjneo/workspace/hecto_deepfake\data/t...   1058    1411   
4  C:/Users/yjneo/workspace/hecto_deepfake\data/t...   1280     720   

   aspect_ratio  brightness  file_size_kb format  
0         0.594      128.49        801.86   None  
1         1.500      129.71       1371.78   None  
2         0.750      109.91       1388.13   None  
3         0.750       92.11        689.61   None  
4         1.778      176.74        282.51   None  


In [6]:
# width/height 분포, brightness 평균과 표준편차, file_size_kb 분포
df.describe()

,width,height,aspect_ratio,brightness,file_size_kb
count,245.000000,245.000000,245.000000,245.000000,245.000000
mean,1888.000000,1754.791837,1.082873,122.026735,2305.114653
std,1801.231901,1338.889368,0.444762,33.395215,4262.440041
min,213.000000,240.000000,0.511000,27.910000,12.130000
25%,800.000000,720.000000,0.684000,102.900000,260.740000
50%,1280.000000,1305.000000,0.801000,119.680000,505.530000
75%,2000.000000,2000.000000,1.500000,143.190000,2264.850000
max,8256.000000,5504.000000,1.900000,211.480000,36734.660000


In [7]:
# 해상도 분포 확인
df['resolution'] = df['width'].astype(str) + "x" + df['height'].astype(str)
df['resolution'].value_counts().head(10)

resolution
1280x720     22
404x720      15
6960x4640    13
5184x3888    12
1024x1536    11
3024x4032     7
640x480       7
768x1024      5
1058x1411     5
1500x2000     5
Name: count, dtype: int64

In [8]:
# 종횡비 이상치 확인
"""
1.0 근처 : 정사각 crop
1.3~1.8 : 원본 프레임 가능성
극단값: 얼굴 crop 실패 가능성
"""
df["aspect_ratio"].value_counts().sort_index()

aspect_ratio
0.511     1
0.525     1
0.561    15
0.566     2
0.594     5
0.600     7
0.602     6
0.625     1
0.666     1
0.667    15
0.668     2
0.673     4
0.674     1
0.684     2
0.726     1
0.727     4
0.749     3
0.750    29
0.751     1
0.757     4
0.758     1
0.759     3
0.761     1
0.786     2
0.797     2
0.798     1
0.799     6
0.801     3
0.814     2
0.853     2
0.903     2
0.944     2
1.109     1
1.205     1
1.288     2
1.333    29
1.334     1
1.345     2
1.375     4
1.395     1
1.398     3
1.399     1
1.497     2
1.500    25
1.505     3
1.506     2
1.778    26
1.784     1
1.897     3
1.899     5
1.900     1
Name: count, dtype: int64

# 동영상 데이터
- 255개

| 항목          | Test Data 관측  | 핵심 위험                 | 필수 Train 전략          | 구현 기준                     |
| ----------- | ------------- | --------------------- | -------------------- | ------------------------- |
| 프레임 수       | 영상 길이·FPS 다양  | 길이/FPS 정보 leakage     | **영상당 고정 N 프레임**     | 16 또는 32, 균등 sampling     |
| Sampling 방식 | 전체 프레임 아님     | temporal bias         | **균등 간격 sampling**   | `np.linspace`             |
| 해상도         | 고해상도·세로 혼재    | resize 왜곡             | **aspect 유지 resize** | short-side 기준             |
| Crop 전략     | 얼굴 위치 불확실     | 배경 bias               | **crop 기반 통일**       | train=random / val=center |
| 밝기 분포       | mean·std 편차 큼 | illumination artifact | **정규화 필수**           | ImageNet mean/std         |
| 조명 변화       | 일부 급격한 변화     | 조명 패턴 학습              | **약한 color jitter**  | brightness/contrast ±0.1  |
| 압축 편차       | 파일 크기 극단적     | 압축 artifact 학습        | **압축 영향 완화**         | jpeg compression aug      |



In [9]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

ROOT_DIR = r"C:/Users/yjneo/workspace/hecto_deepfake/data/test_data"
VIDEO_EXTENSIONS = (".mp4", ".avi", ".mov", ".mkv")

FRAME_STRIDE = 10     # 10프레임마다 샘플
MAX_FRAMES = 200      # 영상당 최대 샘플 프레임 수


In [10]:
video_paths = []
for root, _, files in os.walk(ROOT_DIR):
    for f in files:
        if f.lower().endswith(VIDEO_EXTENSIONS):
            video_paths.append(os.path.join(root, f))

len(video_paths)


255

In [11]:
records = []

for video_path in tqdm(video_paths):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        continue

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    file_size_kb = os.path.getsize(video_path) / 1024
    duration_sec = total_frames / fps if fps > 0 else 0

    frame_idx = 0
    sampled = 0

    brightness_vals = []

    width = height = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % FRAME_STRIDE != 0:
            frame_idx += 1
            continue

        if height is None:
            height, width, _ = frame.shape

        brightness_vals.append(frame.mean())

        sampled += 1
        frame_idx += 1

        if sampled >= MAX_FRAMES:
            break

    cap.release()

    if sampled == 0:
        continue

    records.append({
        "video_path": video_path,
        "fps": round(fps, 2),
        "total_frames": total_frames,
        "duration_sec": round(duration_sec, 2),
        "frame_width": width,
        "frame_height": height,
        "aspect_ratio": round(width / height, 3) if height else 0,
        "brightness_mean": round(np.mean(brightness_vals), 2),
        "brightness_std": round(np.std(brightness_vals), 2),
        "file_size_kb": round(file_size_kb, 2),
        "sampled_frames": sampled
    })

df_video = pd.DataFrame(records)
df_video.head()


100%|██████████| 255/255 [01:39<00:00,  2.56it/s]


,video_path,fps,total_frames,duration_sec,frame_width,frame_height,aspect_ratio,brightness_mean,brightness_std,file_size_kb,sampled_frames
0,C:/Users/yjneo/workspace/hecto_deepfake/data/t...,30.00,93,3.10,1280,720,1.778,99.84,7.12,1493.77,10
1,C:/Users/yjneo/workspace/hecto_deepfake/data/t...,30.00,65,2.17,404,720,0.561,117.44,7.14,399.79,7
2,C:/Users/yjneo/workspace/hecto_deepfake/data/t...,29.97,150,5.00,1024,576,1.778,110.56,0.15,383.94,15
3,C:/Users/yjneo/workspace/hecto_deepfake/data/t...,30.00,100,3.33,1280,720,1.778,184.01,6.41,1127.62,10
4,C:/Users/yjneo/workspace/hecto_deepfake/data/t...,30.00,90,3.00,378,720,0.525,115.70,2.57,428.33,9


In [ ]:
df_video.describe()


,fps,total_frames,duration_sec,frame_width,frame_height,aspect_ratio,brightness_mean,brightness_std,file_size_kb,sampled_frames
count,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000
mean,28.164039,121.831373,4.339961,1433.105882,903.960784,1.552988,134.050235,3.336353,5106.151451,12.447059
std,3.074219,32.620808,1.080188,578.724542,207.844464,0.475500,36.710856,4.421563,9062.662372,3.180301
min,16.000000,64.000000,2.130000,284.000000,540.000000,0.524000,34.970000,0.050000,74.580000,7.000000
25%,24.010000,99.000000,3.330000,1024.000000,720.000000,1.778000,112.755000,1.035000,886.340000,10.000000
50%,30.000000,121.000000,5.000000,1920.000000,1080.000000,1.778000,135.240000,2.030000,1675.160000,13.000000
75%,30.000000,150.000000,5.000000,1920.000000,1080.000000,1.778000,158.240000,4.335000,5025.250000,15.000000
max,32.000000,260.000000,8.670000,1920.000000,1536.000000,1.900000,210.260000,52.510000,71506.650000,26.000000


: 